In [1]:
from modsim import *

In [2]:
from pandas import read_html

In [3]:
filename = 'data/World_population_estimates.html' 
tables = read_html (filename, 
                    header = 0, 
                    index_col = 0, 
                    decimal = 'M')

In [4]:
table2 = tables[2]

In [5]:
table2.columns = ['census', 'prb', 'un', 'maddison', 'hyde', 'tanton', 'biraben', 'mj', 'thomlinson', 'durand', 'clark']

In [6]:
census = table2.census

In [7]:
def plot_estimates():
    un = table2.un / 1e9
    census = table2.census / 1e9
    
    plot(census, ':', color='darkblue', label = 'US Census')
    plot(un, '--', color='green', label = 'UN DESA')
    
    decorate(xlabel='Year',
             ylabel='World population (billion)')

In [8]:
un = table2.un / 1e9

In [9]:
census = table2.census / 1e9

In [10]:
first_year = census.index[0]
last_year = census.index[-1]

In [11]:
total_growth = census[last_year] - census[first_year]
elapsed_time = last_year - first_year
annual_growth = total_growth / elapsed_time

In [12]:
results = TimeSeries()

In [13]:
results[1950] = census[1950]

In [14]:
for t in linrange(1950,2015):
    results[t+1] = results[t] + annual_growth

In [15]:
t0 = census.index[0]
t_end = census.index[-1]
total_growth = census[t_end] - census[t0]
elapsed_time = t_end - t0
annual_growth = total_growth / elapsed_time

In [16]:
system = System(t0=first_year, 
                t_end=last_year, 
                p0=census[first_year],
                annual_growth=annual_growth)

In [17]:
def run_simulation1(system):
    results=TimeSeries()
    results[system.t0] = system.p0
    for t in linrange(system.t0, system.t_end):
        results[t+1] = results[t] + system.annual_growth
    system.results = results

In [18]:
def plot_results(system):
    newfig()
    plot_estimates()
    plot(system.results, '--', color='gray', label='model')
    decorate(xlabel='Year',
            ylabel='World population (billion)',
            ylim=[0,8])

In [19]:
run_simulation1(system)
plot_results(system)

In [20]:
def run_simulation2(system):
    results = TimeSeries()
    results[system.t0] = system.p0
    for t in linrange(system.t0, system.t_end):
        births = system.birth_rate * results[t]
        deaths = system.death_rate * results[t]
        results[t+1] = results[t] + births - deaths
    system.results = results

In [21]:
system.death_rate = 0.01
system.birth_rate = 0.027

In [22]:
def update_func1(pop, t, system):
    births = system.birth_rate * pop
    deaths = system.death_rate * pop
    return pop + births - deaths

In [23]:
def run_simulation(system, update_func):
    results = TimeSeries()
    results[system.t0] = system.p0
    for t in linrange(system.t0, system.t_end):
        results[t+1] = update_func(results[t], t, system)
    system.results = results

In [24]:
run_simulation(system,update_func1)

In [25]:
system.alpha = system.birth_rate - system.death_rate

In [26]:
def update_func1b(pop, t, system):
    net_growth = system.alpha * pop
    return pop + net_growth

In [27]:
run_simulation(system, update_func1b)

In [28]:
pop = census[t]

In [29]:
net_growth = system.alpha * pop

In [30]:
def update_func2(pop, t, system):
    net_growth = system.alpha * pop + system.beta * pop**2
    return pop + net_growth

In [31]:
system.alpha = system.birth_rate - system.death_rate

In [32]:
system.alpha = 0.025
system.beta = -0.0018


In [33]:
net_growth = system.alpha * pop + system.beta * pop**2

In [34]:
run_simulation(system, update_func2)

In [35]:
pop_array = linspace(0.001, 15, 100)
net_growth_array = (system.alpha * pop_array + system.beta * pop_array**2)

In [36]:
def carrying_capacity(system):
    K = -system.alpha / system.beta
    return K
sys1 = System(alpha=0.025, beta=-0.0018)
pop = carrying_capacity(sys1)
print(pop)


13.8888888889


In [39]:
newfig()
run_simulation(system,update_func2)
plot_results(system)